<a href="https://colab.research.google.com/github/mariolyon/llm-zoomcamp/blob/main/homework03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install -U minsearch qdrant_client tqdm -q

In [ ]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')


In [ ]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total)
    }

In [ ]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

def search_minsearch_text(q):
    filter_dict = {"course": q['course']}
    boost_dict = {'question': 1.5, 'section': 0.1}

    query = q['question']
    return index.search(query, filter_dict=filter_dict, boost_dict=boost_dict, num_results=5)


In [6]:
print(evaluate(ground_truth, search_minsearch_text))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.848714069591528, 'mrr': 0.7288235717887772}


In [7]:
from minsearch import VectorSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)


In [8]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [9]:
def search_minsearch_vector(q):
    filter_dict = {"course": q['course']}

    query = {q['question']}
    vectors = pipeline.transform(query)
    return vindex.search(vectors, filter_dict=filter_dict, num_results=5)

In [10]:
print(evaluate(ground_truth, search_minsearch_vector))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.48173762697212014, 'mrr': 0.3572833369353793}


In [11]:
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)


In [12]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [13]:
print(evaluate(ground_truth, search_minsearch_vector))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.8210503566025502, 'mrr': 0.6717347453353508}


In [14]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


In [15]:
model="jinaai/jina-embeddings-v2-small-en"

In [16]:
from dlt.destinations import qdrant
qd_db = "qdrant_storage"
from google.colab import userdata
qd_api_key = userdata.get('QD_API_KEY')
qd_url =  userdata.get('QD_URL')
qd_destination = qdrant(model=model, url=qd_url, api_key=qd_api_key)

SyntaxError: invalid syntax (ipython-input-3231671964.py, line 6)

In [ ]:
import dlt
@dlt.resource
def homework3_data():
    for document in documents:
        yield doc['question'] + ' ' + doc['text']

In [ ]:
#docker run -p 6333:6333 -p 6334:6334 -v "$(pwd)/qdrant_storage:/qdrant/storage:z" qdrant/qdrant

In [ ]:
qd_collection= "homework3_dataset"
qdrant_pipeline = dlt.pipeline(
    pipeline_name="homework3",
    destination=qd_destination,
    dataset_name=qd_collection,
)
load_info = qdrant_pipeline.run(homework3_data())

In [ ]:
from qdrant_client import QdrantClient
qd_db = "db.qdrant"
client = QdrantClient(host="localhost", port=6333)

In [ ]:
vector_name="fast-jina-embeddings-v2-small-en"

In [ ]:
model_name="jinaai/jina-embeddings-v2-small-en"

In [ ]:
from fastembed import TextEmbedding

def embeddingsFor(docs, model_name):
    model = TextEmbedding(model_name=model_name)
    return list(model.query_embed(docs))[0]

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue
import numpy as np

def search_qdrant(q):
    course = q['course']
    query = q['question']

    # Create an embedding for the entire query string.
    query_vector = embeddingsFor([query], model_name)

    query_filter=Filter(
        must=[FieldCondition(key="course", match=MatchValue(value=course))]
    )

    # Use the generated vector in the search query.
    result = client.query_points(collection_name=qd_collection,
                               query_filter=query_filter,
                               query=query_vector,
                               using=vector_name,
                               with_payload= True,
                               limit=5)

    payloads = [point.payload for point in result.points]

    return payloads

In [ ]:
evaluation = evaluate(ground_truth, search_qdrant)

In [ ]:
print(evaluation)